### Minneapolis Inst. of Arts collection

In [1]:
from glob import glob
from scipy import ndimage as ndi
import scipy.fftpack 
from skimage import data
from skimage.util import img_as_float
from skimage.filter import gabor_kernel
from PIL import Image
from object_parsing.src.image_objects import make_a_blank
import numpy as np
from os import path
from experimental_design.src.experiments import make_design_matrix, fmri_experiment
%pylab inline

Populating the interactive namespace from numpy and matplotlib


#### parameters

In [2]:
savedir = '/Users/tnaselar/Data/Presentation/predictive.coding.match.nonmatch/art/'

##image parameters
STIMULUS_PIXELS = 768
VIEWING_ANGLE = 22.1 ##edge-to-edge of 768pix stim
PIXELS_PER_DEGREE = STIMULUS_PIXELS/VIEWING_ANGLE
print 'pixels per deg. %0.3f' %(PIXELS_PER_DEGREE)
IMAGES = 24
isi_luminance = 137

##timing parameters
PHASE_CYCLES_PER_TR = 1 ##number of frames per tr
TR = 1.5
LOOPS = 2 ##number of times each grating shown in a run.
ISI_LAM = 0.4
FADE_IN_FRAMES = 8
FADE_OUT_FRAMES = 8
RUNS = 8 ##
RUN_DURATION = ((IMAGES*(2+ISI_LAM))*LOOPS + FADE_IN_FRAMES+FADE_OUT_FRAMES)*TR/60.
print '%d unique images will be shown %d times per run' %(IMAGES,LOOPS)
print 'average run duration (min): %0.3f' %(RUN_DURATION)
print 'total time across runs (min): %0.3f, total reps of each image across runs: %d' %(RUN_DURATION*RUNS, LOOPS*RUNS)
print 'given a TR of %0.3f, movie frame rate should be (Hz): %0.4f' %(TR, PHASE_CYCLES_PER_TR/TR)

pixels per deg. 34.751
24 unique images will be shown 2 times per run
average run duration (min): 3.280
total time across runs (min): 26.240, total reps of each image across runs: 16
given a TR of 1.500, movie frame rate should be (Hz): 0.6667


####stimuli

In [3]:
pics = glob('/Users/tnaselar/Data/Presentation/mia.art/pictures/*.png')
np.random.shuffle(pics)
pics = pics[:IMAGES]
for pic_name in pics:
    img = Image.open(pic_name).resize((STIMULUS_PIXELS, STIMULUS_PIXELS))
    pic_name = path.basename(pic_name)
    img.save(path.join(savedir,pic_name),'png')

In [4]:
isi_screen = make_a_blank(STIMULUS_PIXELS,isi_luminance,'RGB')
isi_screen.save(path.join(savedir,'isi_screen.png'),'png')

####experimental design

In [5]:
def frame_name(savedir,run,filetype='.png'):
    return path.join(savedir,'mia_art_run_%0.2d.%s' %(run,filetype))

stim_list = []
for ss in map(path.basename, glob(path.join(savedir,'*.png'))):
    stim_list.append([ss])
stim_list.insert(0,['isi_screen.png']) ##<<blank stim always goes first in stim_list
stim_list

[['isi_screen.png'],
 ['100525_mia341_9487.png'],
 ['100624_mia341_9510.png'],
 ['isi_screen.png'],
 ['mia_1005776.png'],
 ['mia_10256b.png'],
 ['mia_11424b.png'],
 ['mia_2003711.png'],
 ['mia_27b.png'],
 ['mia_434b.png'],
 ['mia_48264a.png'],
 ['mia_5002069.png'],
 ['mia_5002155.png'],
 ['mia_5008467.png'],
 ['mia_5009852.png'],
 ['mia_5012616.png'],
 ['mia_5018721.png'],
 ['mia_5019110.png'],
 ['mia_5021260.png'],
 ['mia_5023731.png'],
 ['mia_6007261.png'],
 ['mia_6216b.png'],
 ['mia_7785b.png'],
 ['mia_7831b.png'],
 ['PCD19990121_1178_IMG0042.png'],
 ['PCD19990622_0051_IMG0009.png']]

In [6]:
for rr in range(RUNS):
    dm = make_design_matrix(IMAGES,LOOPS,blank_states=[FADE_IN_FRAMES,ISI_LAM,FADE_OUT_FRAMES],seconds_per_state=TR)
    one_run = fmri_experiment(dm,stim_list,seconds_per_state=TR,vols_per_state = 1)
    one_run.print_frame_list(frame_name(savedir,rr,filetype='txt'))

length of experiment: 136 states and 204.000000 seconds = 3.400000 mintues
length of experiment: 128 states and 192.000000 seconds = 3.200000 mintues
length of experiment: 136 states and 204.000000 seconds = 3.400000 mintues
length of experiment: 131 states and 196.500000 seconds = 3.275000 mintues
length of experiment: 131 states and 196.500000 seconds = 3.275000 mintues
length of experiment: 130 states and 195.000000 seconds = 3.250000 mintues
length of experiment: 134 states and 201.000000 seconds = 3.350000 mintues
length of experiment: 127 states and 190.500000 seconds = 3.175000 mintues
